In [ ]:
!pip install faiss-cpu
!pip install transformers
!pip install pyngrok
!pip install googletrans==3.1.0a0

In [ ]:
from flask import Flask, request, jsonify, render_template, session
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import faiss
import joblib
import numpy as np
import torch
from pyngrok import ngrok
from googletrans import Translator

In [ ]:
app = Flask(__name__)
app.secret_key = '123456'

In [ ]:
# Función para generar embeddings
def generar_embeddings(textos, tokenizer, modelo):
    embeddings_list = []
    for texto in textos:
        inputs = tokenizer(texto, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = modelo.encoder(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        embeddings_list.append(embeddings)
    return np.array(embeddings_list)

In [ ]:
# Función para buscar la pregunta más similar
def buscar_pregunta_similar(input_question, tokenizer, model, index, preguntas, respuestas, k=1):
    input_embedding = generar_embeddings([input_question], tokenizer, model)
    D, I = index.search(input_embedding.astype('float32'), k)
    return [preguntas[i] for i in I[0]], [respuestas[i] for i in I[0]]

In [ ]:
# Función para generar respuesta
def generar_respuesta(modelo, tokenizer, pregunta, respuesta):
    input_text = f"Pregunta: {pregunta}\nRespuesta: {respuesta}\nGenera una respuesta más detallada:"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = modelo.generate(**inputs, max_length=150, num_return_sequences=1, temperature=0.7)
    respuesta_generada=tokenizer.decode(outputs[0], skip_special_tokens=True)
    #return tokenizer.decode(outputs[0], skip_special_tokens=True)

    #traduccion
    translator=Translator()
    respuesta_traducida=translator.translate(respuesta_generada, dest="es").text
    return respuesta_traducida

try:
    # Cargar el modelo al iniciar la aplicación
    print("Cargando el modelo...")
    loaded_data = joblib.load('/content/chatbot_model.joblib')
    model = loaded_data['model']
    tokenizer = loaded_data['tokenizer']
    index = loaded_data['faiss_index']
    preguntas = loaded_data['preguntas']
    respuestas = loaded_data['respuestas']
    print("Modelo cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el modelo: {str(e)}")
    raise

Cargando el modelo...


/usr/local/lib/python3.10/dist-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


Modelo cargado exitosamente.


In [ ]:
@app.route('/chat')
def chat():
    return render_template('chat.html', conversacion=session.get('conversacion', []))

In [ ]:
@app.route('/', methods=['GET', 'POST'])
def home():
    if 'conversacion' not in session:
        session['conversacion'] = []

    if request.method == 'POST':
        pregunta = request.form['pregunta']
        preguntas_similares, respuestas_similares = buscar_pregunta_similar(pregunta, tokenizer, model, index, preguntas, respuestas)
        respuesta_detallada = generar_respuesta(model, tokenizer, preguntas_similares[0], respuestas_similares[0])

        session['conversacion'].append({'pregunta': pregunta, 'respuesta': respuesta_detallada})
        session.modified = True

        return jsonify({'pregunta': pregunta, 'respuesta': respuesta_detallada})

    return render_template('index2.html', conversacion=session.get('conversacion', []))

In [ ]:
if __name__ == '__main__':
    # Reemplaza 'your_auth_token' con tu token de autenticación de ngrok
    ngrok.set_auth_token('INGRESA_TU_TOKEN')

     # Abre un túnel HTTP a través del puerto 5000
    public_url = ngrok.connect(5000, "http")
    print(f'Aplicación disponible en: {public_url}')

    # Ejecuta el servidor Flask
    app.run(port=5000)

Aplicación disponible en: NgrokTunnel: "https://aaf4-34-72-15-56.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [30/Aug/2024 21:36:56] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Aug/2024 21:36:56] "GET /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Aug/2024 21:36:57] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [30/Aug/2024 22:09:25] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Aug/2024 22:09:40] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Aug/2024 22:10:06] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - 